## 1. Import Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
from glob import glob
from sklearn.model_selection import train_test_split

2024-05-16 16:42:11.751981: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 16:42:11.752089: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 16:42:11.885168: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 2. Define Image Size and Batch Size:

In [2]:
IMAGE_SIZE = 224 # define the image size for all images (height and Width = 224 X 224)
BATCH_SIZE = 32  # At the time of training 64 images will be there at a time

# 3. Set Folder Paths:

In [3]:
base_dir = "/kaggle/input/tomatoleaf/tomato"
train_dir = "/kaggle/input/tomatoleaf/tomato/train"
validation_dir = "/kaggle/input/tomatoleaf/tomato/val"

# 4. Get the Number of Classes from Folder Names:

In [4]:
folders = glob(train_dir + '/*')
print(len(folders))

num_classes = len(folders)

10


# 5. Load ResNet50 Model:

In [5]:
resnet_model = ResNet50(weights='imagenet', include_top=False)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# 6. Modify the output layer

In [6]:
x = resnet_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)  # Add your custom layers
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(512, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.3)(x)  # Adding dropout for regularization
x = Dense(256, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.3)(x)  # Adding dropout for regularization
predictions = Dense(num_classes, activation='sigmoid')(x)  # num_classes is the number of classes in your dataset

# 7. Freeze Convolutional Layers (Optional Fine-Tuning):

In [7]:
# Unfreeze the last few layers of the base model
for layer in resnet_model.layers[-10:]:
    layer.trainable = True

# 8. Create a new model with the new output layer

In [8]:
model = Model(inputs=resnet_model.input, outputs=predictions)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, None,      │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,472 │ conv1_pad[0][0]   │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, None,      │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, None,      │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, None,      │          0 │ conv2_block1_2_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, None,      │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, None,      │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, None,      │      1,024 │ conv2_block1_3_c

 Total params: 26,344,586 (100.50 MB)

 Trainable params: 26,291,466 (100.29 MB)

 Non-trainable params: 53,120 (207.50 KB)

# 9. Compile the Model (Initial Training):

In [9]:
# compile the mode
rmsprop_optimizer = RMSprop(learning_rate=0.0001, rho=0.9)
model.compile(optimizer=rmsprop_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 10. Data Augmentation for Training and Validation:

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split = 0.2  # 20% for validation
)
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2 # 20% for validation
)
# for acutual val folder data
test_datagen = ImageDataGenerator(rescale=1./255)

# 11. Prepare Data Generators:

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
)

test_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical'
)

Found 10000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


# 12. Early Stopping and Learning Rate Reduction:

In [12]:
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, min_lr=0.0001)
model_checkpoint = ModelCheckpoint('tomato_resnet50_model_sigmoid_update1.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=1)

# 13. Train the Model (Initial):

In [13]:
epoch = 20
history = model.fit(
    train_generator,
    epochs=epoch,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1715877845.011121     102 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1715877845.103466     102 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 545ms/step - accuracy: 0.5693 - loss: 1.2617

W0000 00:00:1715878020.175639     101 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



Epoch 1: val_loss improved from inf to 25.98742, saving model to tomato_resnet50_model_sigmoid_update1.weights.h5
313/313 ━━━━━━━━━━━━━━━━━━━━ 262s 640ms/step - accuracy: 0.5699 - loss: 1.2599 - val_accuracy: 0.1000 - val_loss: 25.9874 - learning_rate: 1.0000e-04
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step - accuracy: 0.9317 - loss: 0.2496
Epoch 2: val_loss improved from 25.98742 to 6.54956, saving model to tomato_resnet50_model_sigmoid_update1.weights.h5
313/313 ━━━━━━━━━━━━━━━━━━━━ 140s 438ms/step - accuracy: 0.9317 - loss: 0.2496 - val_accuracy: 0.0965 - val_loss: 6.5496 - learning_rate: 1.0000e-04
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step - accuracy: 0.9501 - loss: 0.1886
Epoch 3: val_loss improved from 6.54956 to 5.87991, saving model to tomato_resnet50_model_sigmoid_update1.weights.h5
313/313 ━━━━━━━━━━━━━━━━━━━━ 139s 437ms/step - accuracy: 0.9501 - loss: 0.1886 - val_accuracy: 0.3505 - val_loss: 5.8799 - learning_rate: 1.0000e-04
Epoch 4/20
313/313 ━━━━━━━

# 14. Evaluate the Model after Initial Training:

In [14]:
score = model.evaluate(test_generator)
print("Test loss (initial):", score[0])
print("Test accuracy (initial):", score[1])

32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 348ms/step - accuracy: 0.9683 - loss: 0.1313
Test loss (initial): 0.14436355233192444
Test accuracy (initial): 0.9509999752044678


# 15. Get the classification Report

In [15]:
import numpy as np
from sklearn.metrics import classification_report
class_labels = ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 
                'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
                'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus',
                'Tomato___healthy']

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_labels = test_generator.classes
report = classification_report(true_labels, predicted_classes, target_names=class_labels)
print(report)

 5/32 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step

W0000 00:00:1715879323.707979     102 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 175ms/step
                                               precision    recall  f1-score   support

                      Tomato___Bacterial_spot       0.92      0.99      0.95       100
                        Tomato___Early_blight       0.99      0.93      0.96       100
                         Tomato___Late_blight       0.93      0.99      0.96       100
                           Tomato___Leaf_Mold       0.99      1.00      1.00       100
                  Tomato___Septoria_leaf_spot       0.99      0.96      0.97       100
Tomato___Spider_mites Two-spotted_spider_mite       0.90      1.00      0.95       100
                         Tomato___Target_Spot       1.00      0.90      0.95       100
       Tomato___Tomato_Yellow_Leaf_Curl_Virus       0.85      0.90      0.87       100
                 Tomato___Tomato_mosaic_virus       0.98      0.85      0.91       100
                             Tomato___healthy       1.00      0.99      0.99       100



# 16. Update the weights of the model

In [16]:
model.load_weights('/kaggle/working/tomato_resnet50_model_sigmoid_update1.weights.h5')